<a href="https://colab.research.google.com/github/youseefmoemen/Neural-Network/blob/main/Sup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-1 * z))
def sigmoid_grad(z):
  t = sigmoid(z)
  return t * (1 - t)
def leakyRelu(z):
  return np.where(z >= 0, z, z * 0.3)
def leakyRelu_grad(z):
  return np.where(z >= 0, 1, 0.3)
def softmax(z):
  e = np.exp(z)
  return e / np.sum(e)
def softmax_grad(z):
    I = np.eye(z.shape)
    return np.dot((I - softmax(z)).T, softmax(z))
def tanh(z):
  return np.tanh(z)
def tanh_grad(z):
  return 1 - np.power(tanh(z), 2)

In [ ]:
Activations = {
    'sigmoid': sigmoid,
    'leakyRelu': leakyRelu,
    'softmax': softmax,
    'tanh': tanh
}
Activations_grad = {
    'sigmoid': sigmoid_grad,
    'leakyRelu': leakyRelu_grad,
    'softmax': softmax_grad,
    'tanh': tanh_grad
}

In [296]:
from sklearn.datasets import fetch_openml
import math

In [297]:
mnist = fetch_openml('mnist_784', version = 1)

In [331]:
from sklearn.model_selection import StratifiedShuffleSplit

In [332]:
sss = StratifiedShuffleSplit(n_splits = 1, train_size=10000, test_size=1000)

In [450]:
for train_index, test_index in sss.split(mnist.data, mnist.target):
  X_train = mnist.data.iloc[train_index] 
  y_train = mnist.target[train_index]
  X_test = mnist.data.iloc[test_index] 
  y_test = mnist.target[test_index]

In [451]:
def clip_image(img, clipping_size = 0):
  img = img.reshape((28, 28))
  img = np.delete(img, range(clipping_size), 0)
  img = np.delete(img, range(clipping_size), 1)
  return img.flatten()

In [452]:
def momentum(img, step_size, n_steps, clipping_size):
  img = img.reshape((28 - clipping_size, 28-clipping_size))
  momentums = []
  for i in range(n_steps):
    x = np.array(list(range(i*step_size, (i+1)*step_size)))
    for j in range(n_steps):
      pixels = img[j*step_size: (j+1)*step_size, i*step_size: (i+1) * step_size]
      y =  np.array(list(range(j*step_size, (j+1)*step_size))).reshape(step_size,1)
      area = np.sum(np.sum(pixels))
      x_c = np.sum(np.sum(x.T * pixels)) / (area + 1e-10)
      y_c = np.sum(np.sum(y * pixels)) / (area + 1e-10)
      momentums.append((x_c, y_c))
  return momentums

In [453]:
def reduction(data = None, n_momentums = 9):
  n_steps = math.floor(np.sqrt(n_momentums))
  clipping_size = 28 % n_steps
  step_size = (28 - clipping_size) // n_steps
  data = np.apply_along_axis(clipping_size = clipping_size
                      , func1d = clip_image, axis = 1, arr = data)
  momentums = np.apply_along_axis(step_size = step_size, n_steps = n_steps,
                                  clipping_size= clipping_size,
                                  func1d = momentum, axis = 1, arr = data)
  return data, momentums

In [454]:
X_train = reduction(np.array(X_train.copy()), n_momentums = 9)[1].reshape(10000, 18)

In [406]:
class Layer():
  def __init__(self, n_neurons, activation):
    self.n_neurons = n_neurons
    self.w = None
    self.activation = Activations[activation]
    self.activation_grad = Activations_grad[activation]
  
  def initialize(self, prev):
    self.w = np.random.normal(size=(self.n_neurons, prev))

  def compute(self, Oi):
    Netj = np.dot(Oi, self.w.T)  # W * Oi
    Oj = self.activation(Netj)
    return Netj, Oj
  
  def grad(self, delta, w_next, Oi):
    Netj = self.compute(Oi)[0]
    partial = np.dot(delta, w_next)
    delta = np.multiply(partial, self.activation_grad(Netj))
    Dw = np.dot(delta.T, Oi)
    return Dw, delta

  def update(self, learning_rate, Dw):
    hold = self.w.shape
    self.w  = self.w  -  learning_rate * Dw

In [407]:
class OutputLayer(Layer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
  
  def grad(self, target, Oi):
    Netj, Oj = self.compute(Oi)
    delta = np.multiply(-1* (1/target.shape[0]) * (target - Oj), self.activation_grad(Netj))
    Dw = np.dot(delta.T, Oi)
    return Dw, delta

In [447]:
class NeuralNetwork():
  def __init__(self, layers, input_shape):
    self.layers = layers
    self.O_ALL = None
    self.input_shape = input_shape
    self.layers[0].initialize(self.input_shape)
    for i in range(1, len(self.layers)):
      self.layers[i].initialize(self.layers[i-1].n_neurons)
  
  def predict(self, X):
    Oi = X
    self.O_ALL = []
    self.O_ALL.append(X)
    for layer in self.layers:
      Oi = layer.compute(Oi)[1]
      self.O_ALL.append(Oi)
    return Oi
  

  def loss(self, Oj, y):
    E = (2 / y.shape[0]) * np.sum((y - Oj) ** 2)
    return E
  
  def accuracy(self, y_pred, y_true):
    return np.sum(np.equal(np.argmax(y_true, axis = 1), np.argmax(y_pred, axis = 1))) / len(y_true)

  def train(self, X_train, y_train, learning_rate, epochs, 
            X_val = None, y_val = None):
    history = []
    for i in range(epochs):
        y_pred = self.predict(X_train)
        print(f'iteration {i} accuracy {self.accuracy(y_pred, y_train)}' ,
              f'loss {self.loss(y_train, y_pred)}')
        for j in reversed(range(len(self.layers))):
          if j == len(self.layers) - 1:
            Dw, delta = self.layers[j].grad(y_train, self.O_ALL[-2])
          else:
            Dw, delta = self.layers[j].grad(delta, hold, self.O_ALL[j])
          hold = self.layers[j].w.copy()
          self.layers[j].update(learning_rate, Dw)

In [448]:
from sklearn.datasets import make_moons
X_train, y_train = make_moons()

In [455]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

In [460]:
layers = [
    Layer(100, 'leakyRelu'),
    Layer(50, 'leakyRelu'),
    Layer(30, 'leakyRelu'),
    OutputLayer(10, 'leakyRelu')
]

In [461]:
nn = NeuralNetwork(layers, 18)

In [462]:
print(X_train.shape, y_train.shape)

(10000, 18) (10000, 10)


In [ ]:
nn.train(X_train, y_train, 1e-8, 1000)

In [ ]:
for i in nn.layers:
  print(i.w.shape)

(30, 2)
(10, 30)
(3, 10)
(2, 3)
